<a href="https://colab.research.google.com/github/IshaSarangi/Edureka_Notes/blob/main/Edureka_MT_using_OpenAI_and_Groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/drive/1BFxhkw7ykCi9b-ZM7QjDUh2NAVmpvRC5?usp=sharing

Translation English---->Hindi

Dataset: IIT Bombay English-Hindi Corpus
https://www.cfilt.iitb.ac.in/iitb_parallel/

In [1]:
!pip install openai datasets nltk transformers sentencepiece

In [2]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [3]:
import pandas as pd
from datasets import load_dataset
from openai import OpenAI
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
#Load Dataset
dataset = load_dataset('cfilt/iitb-english-hindi')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [5]:
#Take sample for demo
df = pd.DataFrame(dataset['test'][:20])
print(df)

                                          translation
0   {'en': 'A black box in your car?', 'hi': 'आपकी...
1   {'en': 'As America's road planners struggle to...
2   {'en': 'The devices, which track every mile a ...
3   {'en': 'The usually dull arena of highway plan...
4   {'en': 'Libertarians have joined environmental...
5   {'en': 'The tea party is aghast.', 'hi': 'चाय ...
6   {'en': 'The American Civil Liberties Union is ...
7   {'en': 'And while Congress can't agree on whet...
8   {'en': 'They are exploring how, over the next ...
9   {'en': 'Thousands of motorists have already ta...
10  {'en': 'This really is a must for our nation.'...
11  {'en': '"It is not a matter of something we mi...
12  {'en': 'There is going to be a change in how w...
13  {'en': 'The technology is there to do it.', 'h...
14  {'en': 'The push comes as the country's Highwa...
15  {'en': 'Americans don't buy as much gas as the...
16  {'en': 'Cars get many more miles to the gallon...
17  {'en': 'The federal tax 

In [6]:
df['English'] = df['translation'].apply(lambda x: x['en'])
df['Hindi_ref'] = df['translation'].apply(lambda x: x['hi'])
df = df[['English', 'Hindi_ref']]
print(df)

                                              English  \
0                            A black box in your car?   
1   As America's road planners struggle to find th...   
2   The devices, which track every mile a motorist...   
3   The usually dull arena of highway planning has...   
4   Libertarians have joined environmental groups ...   
5                            The tea party is aghast.   
6   The American Civil Liberties Union is deeply c...   
7   And while Congress can't agree on whether to p...   
8   They are exploring how, over the next decade, ...   
9   Thousands of motorists have already taken the ...   
10              This really is a must for our nation.   
11  "It is not a matter of something we might choo...   
12  There is going to be a change in how we pay th...   
13                  The technology is there to do it.   
14  The push comes as the country's Highway Trust ...   
15   Americans don't buy as much gas as they used to.   
16            Cars get many mor

In [7]:
client = OpenAI()
def translate_openai(text):
    response = client.chat.completions.create(
        model = 'gpt-4o-mini',
        messages=[
                {"role": "system","content":"Please Translate English to Hindi."},
                {"role": "user","content":text}
            ]
        )
    return response.choices[0].message.content.strip()


In [8]:
df['Hindi_openai'] = df['English'].apply(translate_openai)
print(df['Hindi_openai'])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

###Compare with MarianMT - English ---> Hindi Model

In [9]:
from transformers import MarianMTModel, MarianTokenizer
from sentence_transformers import SentenceTransformer, util

model_name = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate_marianmt(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens = True)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [10]:
df['Hindi_marianmt'] = df['English'].apply(translate_marianmt)
print(df['Hindi_marianmt'])

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

0                          अपनी कार में एक ब्लैक बॉक्स?
1     के रूप में अमेरिका के सड़क योजनार एक टूटी हुई ...
2     ये उपकरण, जो हर मील का ट्रैक करते हैं एक मोटर ...
3     आम तौर पर सड़क योजना के ठंडे रंग में बहस और रं...
4     कुछ देशों में ऐसे भी लोग हैं जो एक - दूसरे को ...
5                               चाय पार्टी कम हो गई है.
6     अमरीकी नागरिक संघ को भी अलग - अलग निजी मामलों ...
7     और जबकि कांग्रेस आगे बढ़ने पर सहमत नहीं हो सकत...
8     वे कल्पना कर रहे हैं कि अगले दस सालों के दौरान...
9     हज़ारों मोटर - चालकों ने पहले ही ब्लैक बक्से ल...
10       यह वास्तव में हमारी जाति के लिए एक होना चाहिए.
11    "यह कोई बात नहीं है जो हम करने के लिए चुन सकते...
12    हम इन कर का भुगतान कैसे कर रहे हैं में एक परिव...
13                      प्रौद्योगिकी यह करने के लिए है.
14    धक्का देश के उच्चवे पर भरोसा निधि के रूप में आ...
15    अमरीकी उतना गैस नहीं खरीदते जितना वे इस्तेमाल ...
16        झूठ बोलने के लिए कार को और अधिक मील मिलता है.
17    संघीय कर अपने आप, 184 सेंट प्रति सेंट, 20 

In [11]:
import requests
import os

from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
API_Key = os.environ['GROQ_API_KEY']

In [12]:
#define API endpoint

def translate_groq(text):
  url="https://api.groq.com/openai/v1/chat/completions"

  #Set header
  headers={
      "Authorization":f"Bearer {API_Key}",
      "Content-Type":"application/json"
    }

    #Create request
  data={
        "model":"Gemma2-9b-It",
        "messages":[
        {"role":"system","content":"You are a translator."},
        {"role":"user","content":f"Translate this to Hindi: {text}"}
        ]
    }

  resp=requests.post(url,headers=headers,json=data)

  return resp.json()['choices'][0]['message']['content']

In [13]:
df['Hindi_groq'] = df['English'].apply(translate_groq)

In [14]:
print(df['Hindi_groq'])

0                       आपके कार में एक ब्लैक बॉक्स? \n
1     चूँकि अमेरिका के सड़क योजनाकार अपने क्षतिग्रस्...
2     ये उपकरण जो हर माइल की जानकारी एक ड्राइवर द्वा...
3     प्रतिष्ठित सडक़ योजना क्षेत्र में अचानक तीव्र ब...
4     यहाँ, यह वाक्य का हिंदी अनुवाद है:\n\n"स्वतंत्...
5     The translation of "The tea party is aghast" i...
6     अमेरिकन सिविल लिबर्टीज यूनियन भी गहरी चिंता जत...
7     और जबकि कांग्रेस इस बात पर सहमति नहीं कर पाई क...
8     वे इस बात का पता लगा रहे हैं कि अगले दशक में व...
9     हजारों चालक पहले से ही काले बॉक्स ले लिए हैं, ...
10    यह हमारे राष्ट्र के लिए बहुत जरूरी है। \n\n(Ye...
11    "यह कुछ ऐसा नहीं है जो हम चुनने को तैयार हों,"...
12    इन करों का भुगतान करने के तरीके में बदलाव होने...
13    तकनीक उसके लिए मौजूद है। \n\n(Teckniology uske...
14    यह दबाव तब आ रहा है जब देश का राजमार्ग विश्वास...
15    अमेरिकी लोग पहले जितना पेट्रोल खरीदते थे, अब उ...
16    कारें एक गैलन में अब कई अधिक मील देती हैं। \n\...
17    फ़ेडरल टैक्स, जो प्रति गैलन 18.4 सेंट होता

###Evaluate Transactions using BLEU Score

In [15]:
references=[[ref.split()] for ref in df['Hindi_ref']]

#BLEU score OpenAI
cand_openai=[pred.split() for pred in df['Hindi_openai']]
bleu_openai=corpus_bleu(references,cand_openai)
print(f"BLEU Score OpenAI: {bleu_openai}")

KeyError: 'Hindi_openai'

In [16]:
#BLEU score for MarianMT
cand_marianmt = [pred.split() for pred in df['Hindi_marianmt']]
bleu_marianmt = corpus_bleu(references, cand_marianmt)
print(f"BLEU Score MarianMT: {bleu_marianmt}")

BLEU Score MarianMT: 0.061927262276044164


In [18]:
#BLEU score for Groq
cand_groq=[pred.split() for pred in df['Hindi_groq']]
bleu_groq=corpus_bleu(references,cand_groq)
print(f"BLEU Score Groq: {bleu_groq}")

BLEU Score Groq: 0.04613461986341447
